In [1]:
import pandas as pd
import spacy
import emoji
from nltk.corpus import stopwords
from collections import Counter
import numpy as np
from Twitterpreprocessor import TwitterPreprocessor

In [2]:
data = pd.read_csv("./Data/data+descvf.csv")


In [5]:

nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))
def is_bot_profile(profile_description):
    # remove emojis
    profile_description = TwitterPreprocessor(str(profile_description)).desc_preprocess().text
    print("profile_description",profile_description)
    # tokenize the text with spaCy
    doc = nlp(profile_description)
    print("doc",doc)
    # remove stop words and punctuation
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop and token.lemma_ not in stop_words]
    print(tokens)
    # count the frequency of different parts-of-speech
    pos_tags = [token.pos_ for token in doc]
    pos_counts = Counter(pos_tags)
    print("pos count",pos_counts)
    # look for patterns typical of bot accounts
    if pos_counts['PRON'] >= 3 and pos_counts['NOUN'] >= 3:
        return True
    elif pos_counts['VERB'] >= 2 and pos_counts['ADJ'] >= 2:
        return True
    else:
        return False
#  "I am a bot that tweets about sports and technology. Follow me for updates!"
profile_description ="I am a bot that tweets about sports and technology. Follow me for updates!"
is_bot = is_bot_profile(profile_description)
print(f"Profile description: {profile_description}")
print(f"Is bot? {is_bot}")


profile_description  am bot that tweets about sports and technology follow me for updates
doc  am bot that tweets about sports and technology follow me for updates
[' ', 'bot', 'tweet', 'sport', 'technology', 'follow', 'update']
pos count Counter({'NOUN': 4, 'PRON': 2, 'VERB': 2, 'ADP': 2, 'SPACE': 1, 'AUX': 1, 'CCONJ': 1})
Profile description: I am a bot that tweets about sports and technology. Follow me for updates!
Is bot? False


In [1]:
import spacy
import emoji
from nltk.corpus import stopwords
from collections import Counter

nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

def is_bot_profile(profile_description):
    # remove emojis
    # profile_description = emoji.get_emoji_regexp().sub(r'', profile_description)
    # lowercase the text
    profile_description = profile_description.lower()
    # tokenize the text with spaCy
    doc = nlp(profile_description)
    print(doc)
    # remove stop words and punctuation
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop and token.lemma_ not in stop_words]
    print(tokens)
    # count the frequency of different parts-of-speech
    pos_tags = [token.pos_ for token in doc]
    print(pos_tags)
    pos_counts = Counter(pos_tags)
    # look for patterns typical of bot accounts
    if pos_counts['PRON'] >= 3 and pos_counts['NOUN'] >= 3:
        return True
    elif pos_counts['VERB'] >= 2 and pos_counts['ADJ'] >= 2:
        return True
    else:
        return False
profile_description = "I am a bot that tweets about sports and technology. Follow me for updates!"
is_bot = is_bot_profile(profile_description)
print(f"Profile description: {profile_description}")
print(f"Is bot? {is_bot}")

i am a bot that tweets about sports and technology. follow me for updates!


AttributeError: 'spacy.morphology.Morphology' object has no attribute 'tag_map'

In [3]:
import nltk
import spacy

from nltk.tokenize import word_tokenize

# Load the dataset
dataset = pd.read_csv("./Data/datafromAPI.csv")

# Clean the text data
dataset['description'] = dataset['description'].apply(lambda x :TwitterPreprocessor(str(x)).desc_preprocess().text)
dataset['description'] = dataset['description'].fillna('')
# Define function to lemmatize the tokens
nlp = spacy.load('en_core_web_sm')
def lemmatize_tokens(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

# Tokenize and lemmatize the text data
dataset['description_tokens'] = dataset['description'].apply(lambda x: lemmatize_tokens(word_tokenize(x)))

# Remove stop words
stop_words = set(nltk.corpus.stopwords.words('english'))
dataset['description_tokens'] = dataset['description_tokens'].apply(lambda x: [word for word in x if word not in stop_words])

C:\Users\Chams\AppData\Local\Temp\ipykernel_16676\774891700.py:7: DtypeWarning: Columns (0,3,4,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("./Data/datafromAPI.csv")


In [5]:
dataset.to_csv('./Data/description_tokens.csv', index=False)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert the list of tokens to a string
dataset['description_tokens'] = dataset['description_tokens'].apply(lambda x: ' '.join(x))


In [8]:
# Add the dense array to the dataset
dataset['description_tokens'] = X_array.tolist()

In [ ]:
dataset.to_csv('./Data/description_tokensv2.csv', index=False)

start prepare desc

In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Load dataset
df = pd.read_csv('./Data/data+descvf.csv')

# Remove irrelevant or duplicated data
df = df.drop(['description',  
         'account_type'], axis=1)

# Remove special characters and punctuation, and convert text to lowercase
df['description'] = df['description'].apply(lambda x :TwitterPreprocessor(str(x)).desc_preprocess().text)

# Tokenize text
df['description'] = df['description'].apply(lambda x: x.split())

# Remove stop words
stop_words = set(stopwords.words('english'))
df['description'] = df['description'].apply(lambda x: [word for word in x if word not in stop_words])

# Stemming
stemmer = PorterStemmer()
df['description'] = df['description'].apply(lambda x: [stemmer.stem(word) for word in x])

# Convert back to string
df['description'] = df['description'].apply(lambda x: ' '.join(x))


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
# Split into training and testing sets
train_size = int(0.8 * len(df))
train_data = df[:train_size]
test_data = df[train_size:]

# Tokenize text inputs
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data['description'])

# Convert text inputs to sequences of integers
train_sequences = tokenizer.texts_to_sequences(train_data['description'])
test_sequences = tokenizer.texts_to_sequences(test_data['description'])

# Pad sequences
max_len = 100
train_data = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')
test_data = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

# Build model
model = Sequential([
    Embedding(10000, 128, input_length=max_len),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit model
history = model.fit(train_data, train_data['label'], epochs=10, validation_data=(test_data, test_data['label']))

# Evaluate model
loss, accuracy = model.evaluate(test_data, test_data['label'])
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')